# Julia MixedModels.jl workshop @ MindCORE

June Choe

12/01/2023

A [MindCORE workshop](https://mindcore.sas.upenn.edu/calendar_event/workshop-introduction-to-mixed-effects-models-in-julia/)

https://tinyurl.com/mindcore-mixedmodels-jl

## What is this workshop about?

It's about two things:

[Julia](https://julialang.org/): A (relatively modern; 5 years since [v1.0](https://julialang.org/blog/2018/08/one-point-zero/)) programming language for scientific computing. The "ju" part of "**ju**pyter notebook".

[MixedModels.jl](https://github.com/JuliaStats/MixedModels.jl): A successor to R's [lme4](https://cran.r-project.org/web/packages/lme4/index.html) in Julia that has been (rather silently?) developing for the last decade.

## Who is this workshop for?

1) You regularly use R and mixed-effects models for research

2) Your models sometimes (if not often) fail to converge :(

3) You're bothered by (2) enough to try something better/different, but not enough to learn an entire new programming language ([Julia](https://julialang.org/)) and its toolkits (ex: VSCode).

4) You've maybe even heard of Julia and/or MixedModels.jl before, and are intrigued by the more modern and powerful approach to working with mixed effects models, including:

- [Simulation-based power analysis](https://repsychling.github.io/MixedModelsSim.jl/stable/simulation_tutorial/)

- [Parametric bootstrapping](https://repsychling.github.io/SMLP2023/bootstrap.html)

- [New ways of visualizing model diagnostics](https://palday.github.io/MixedModelsMakie.jl/dev/api/)


## What will we cover in this workshop?

None of the fancy stuff I just mentioned. Instead, we'll focus more on the practical side of fitting mixed models in Julia while leveraging our existing experience with R/RStudio/lme4. The goal is to come out of this workshop with *just enough* knowledge (and a taste of what you're getting into) so that we're able to explore the more exciting things on our own.

## Why am I (June) the one giving this workshop?

I'm probably not the most qualified person to teach Julia. But I did attend a workshop a few months ago ([SMLP 2023](https://repsychling.github.io/SMLP2023/)) taught by people who do know how, so I'm more of a messenger trying to spread the good word.

# 1) Setup

First, **copy the notebook to your google drive**. Follow the below instructions for setting up.

_Expect around 10 minutes for all setup code to run._

## 1a) Install Julia to Colab

Run the following long code block _once_. This should take around 3 minutes.

Once it completes with a "successfully installed" message, refresh (Ctrl/Cmd + R) the page and move onto (1b).

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.9.4"
JULIA_PACKAGES="IJulia ProgressMeter"
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key)"
fi

Installing Julia 1.9.4 on the current Colab Runtime...
2023-12-02 00:51:35 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.9/julia-1.9.4-linux-x86_64.tar.gz [146163887/146163887] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package ProgressMeter...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.9

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key)


_Above setup code adopted from the [Julia Colab Notebook Template](https://colab.research.google.com/github/ageron/julia_notebooks/blob/master/Julia_Colab_Notebook_Template.ipynb): see the link for additional troubleshooting._

## 1b) Install MixedModels.jl & extras

Takes around 5-7 minutes.

In [ ]:
using Pkg
Pkg.add("MixedModels")
# We'll also be using these other packages for reading/manipulating data:
Pkg.add("CSV")
Pkg.add("DataFrames")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed Calculus ──────────────────── v0.5.1
   Installed IrrationalConstants ───────── v0.2.2
   Installed DualNumbers ───────────────── v0.6.8
   Installed NLopt_jll ─────────────────── v2.7.1+0
   Installed Adapt ─────────────────────── v3.7.1
   Installed Rmath ─────────────────────── v0.7.1
   Installed Scratch ───────────────────── v1.2.1
   Installed HypergeometricFunctions ───── v0.3.23
   Installed Zstd_jll ──────────────────── v1.5.5+0
   Installed StatsFuns ─────────────────── v1.3.0
   Installed JSON3 ─────────────────────── v1.13.2
   Installed NLopt ─────────────────────── v1.0.0
   Installed TableTraits ───────────────── v1.0.1
   Installed BandedMatrices ────────────── v1.2.1
   Installed Arrow ─────────────────────── v2.6.2
   Installed LoggingExtras ─────────────── v1.0.3
   Installed Mocking ───────────────────── v0.7.7
   Installed SentinelArrays ────────────── v1.4.1
 

## 1c) Check the Installation

Check that you have installed Julia v1.9.4

In [ ]:
versioninfo()

Julia Version 1.9.4
Commit 8e5136fa297 (2023-11-14 08:46 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, broadwell)
  Threads: 3 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2
  JULIA_IMAGE_THREADS = 1


Check that you have installed the MixedModels.jl library (+ some others)

In [ ]:
Pkg.status()

Status `~/.julia/environments/v1.9/Project.toml`
  [336ed68f] CSV v0.10.11
  [a93c6f00] DataFrames v1.6.1
  [7073ff75] IJulia v1.24.2
  [ff71e718] MixedModels v4.22.2
  [92933f4c] ProgressMeter v1.9.0


## 1d) Optional: setup on your own machine

(You can try this section on your own while the colab setup code runs!)

1) Install **Julia**: https://julialang.org/downloads/. For troubleshooting, see [this manual](https://dev-juliacn.github.io/downloads/platform.html).

2) Open **RStudio**. From the [terminal tab](https://support.posit.co/hc/en-us/articles/115010737148-Using-the-RStudio-Terminal-in-the-RStudio-IDE#started), check that Julia is installed (ex: run `julia --version`)

3) Open Julia and install the necessary **libraries** using the same code from (1b).

4) Download/clone the [Github repo](https://github.com/yjunechoe/MindCORE-julia-mixedmodels) for this workshop (more instructions to come in Section 4)

# 2) Anatomy of mixed models in Julia

We're skipping the "hello world" in Julia. Let's jump straight to fitting mixed effects models

In [ ]:
# R's `library()` equivalent of loading a package
using MixedModels
# (Ignore) the following lines suppress the printing of progress bars
using ProgressMeter
ProgressMeter.ijulia_behavior(:append);

## 2a) Review from R

In R, there are (at least) three pieces of information you need to fit a linear model:

1) The model to fit - ex: `lm()`, `lmer()`, `glmer()`

2) The formula - ex: `y ~ x + (1 | z)`

3) The data - ex: a data frame like `lme4::sleepstudy`

Example model fit:

```r
library(lme4)
lmer(Reaction ~ Days + (1 | Subject), lme4::sleepstudy)
#> Linear mixed model fit by REML ['lmerMod']
#> Formula: Reaction ~ Days + (1 | Subject)
#>    Data: sleepstudy
#> REML criterion at convergence: 1786.465
#> Random effects:
#>  Groups   Name        Std.Dev.
#>  Subject  (Intercept) 37.12   
#>  Residual             30.99   
#> Number of obs: 180, groups:  Subject, 18
#> Fixed Effects:
#> (Intercept)         Days  
#>      251.41        10.47
```

## 2b) Translations to Julia

The same three pieces of information are required to fit a model in Julia, though they look slightly different. The call follows the general template of:

```julia
fit(
  # <model>,
  # <formula>,
  # <data>
)
```

For example, translating the sleepstudy model in R from above (mind the [warm-up time](https://en.wikipedia.org/wiki/Just-in-time_compilation)):

In [ ]:
sleepstudy = MixedModels.dataset("sleepstudy")

sleep_model = fit(
  MixedModel,                             # Argument 1: The type of model
  @formula(reaction ~ days + (1 | subj)), # Argument 2: model formula
  sleepstudy                              # Argument 3: data to fit the model to
);

sleep_model

Minimizing 2    Time: 0:00:00 ( 0.12  s/it)

Minimizing 14    Time: 0:00:00 (25.65 ms/it)


|             |     Est. |     SE |     z |      p |  σ_subj |
|:----------- | --------:| ------:| -----:| ------:| -------:|
| (Intercept) | 251.4051 | 9.5062 | 26.45 | <1e-99 | 36.0121 |
| days        |  10.4673 | 0.8017 | 13.06 | <1e-38 |         |
| Residual    |  30.8954 |        |       |        |         |


A few things to note about Julia's syntax:

1) Whereas **model types** are _functions_ in R (`lm()`, `lmer()`), model types are _objects_ in Julia, passed as the first argument to `fit()`, like `fit(MixedModels, ...)`

2) Whereas **model formula** simply requires the tilde `~` in R, it must additionally be wrapped in `@formula()` in Julia

3) In notebooks like Colab, printing the model object simple shows the model estimates and statistics in a table. But in the REPL, the model would print like this, which should look more familiar:


In [ ]:
# simulates how the model actually prints in the "console"
println(sleep_model)

Linear mixed model fit by maximum likelihood
 reaction ~ 1 + days + (1 | subj)
   logLik   -2 logLik     AIC       AICc        BIC    
  -897.0393  1794.0786  1802.0786  1802.3072  1814.8505

Variance components:
            Column    Variance Std.Dev.
subj     (Intercept)  1296.8692 36.0121
Residual               954.5279 30.8954
 Number of obs: 180; levels of grouping factors: 18

  Fixed-effects parameters:
──────────────────────────────────────────────────
                Coef.  Std. Error      z  Pr(>|z|)
──────────────────────────────────────────────────
(Intercept)  251.405     9.50618   26.45    <1e-99
days          10.4673    0.801735  13.06    <1e-38
──────────────────────────────────────────────────


## 2c) A few side-by-side examples

**A logistic regression model**:

In R, you use `glmer()` with the appropriate `family`:

```r
glmer(y ~ x + (1 | z), data, family = binomial()) # or "binomial"
```

In Julia, you can just add the family as the fourth positional argument:

```julia
fit(MixedModel, @formula(y ~ x + (1 | z)), data, Bernoulli()) # Binomial() reserved for modelling proportions w/ weights
```

*You can be more specific by using `GeneralizedLinearMixedModel` instead of `MixedModel` but it'll infer the type from the ["family"](https://juliastats.org/MixedModels.jl/stable/optimization/#Generalized-Linear-Mixed-Effects-Models)*

**A zero-correlation  model**:

In R, you use the double bar `||` syntax:

```r
lmer(y ~ x + (1 + x || z), data)
```

In Julia, you wrap the REs in `zerocorr()`:

```julia
fit(MixedModel, @formula(y ~ x + zerocorr(1 + x | z)), data)
```

**Contrast coding**

In R, contrasts are often defined as an attribute of factor columns via `contrasts<-`:

```r
contrasts(data$x) <- contr.sum(2)
lmer(y ~ x + (1 | z), data)
```

In Julia, contrasts are specified separately from the data, via the `contrasts` argument of `fit()`:

```julia
fit(MixedModel, @formula(y ~ x + (1 | z)), data;
    contrasts = Dict(:x => EffectsCoding()))
```

Actually, R has this syntax too. It's just less popular:

```r
lmer(y ~ x + (1 | z), data,
     contrasts = list(x = contr.sum))
```

See the [documentation](https://juliastats.org/StatsModels.jl/stable/contrasts/#StatsModels.HypothesisCoding) for more built-in contrast schemes and manual coding with `HypothesisCoding()`  

*Note that we add the contrasts argument with the semincolon. In Julia, miscellaneous arguments are separted from required arguments by `;`*

**Toggling REML**:

In R, LMEMs are fit with REML by default. You turn it off with:

```r
lmer(y ~ x + (1 | z), data, REML = FALSE)
```

In Julia, REML is FALSE by default. You turn it on with:

```julia
fit(MixedModel, @formula(y ~ x + (1 | z)), data; REML = true)
```

<details>
<summary>Why the difference in REML defaults?</summary>

[From Douglas Bates](https://github.com/RePsychLing/SMLP2023/discussions/24):

<blockquote>

The REML criterion is a hold-over from [earlier days](https://support.sas.com/documentation/cdl/en/statug/63033/HTML/default/viewer.htm#statug_mixed_sect022.htm) when people felt that estimating "variance components" as variances would naturally imply that these estimators were normally, or at least symmetrically, distributed. If that were the case then the mean of the distribution would be a good location measure and we would want to mean of the estimator to be close to the parameter being estimated. This is what an "unbiased estimator" means.

They played with the definition of the likelihood to define "residual" or "restricted" likelihood and these REML estimates, which are not exactly unbiased for these models, but are closer to being unbiased.

However, the whole question is moot because the estimators are quite skewed and the mean is not a good measure of location. The likelihood and likelihood-ratio tests are more cleanly defined for these models. We characterize variability according to multiple evaluations of the model at different parameter values through, e.g. [profiling](https://repsychling.github.io/SMLP2023/profiling.html), instead of trying to do only one fit and a bunch of mathematical approximations.

</blockquote>
</details>

If we refit our sleepstudy model with REML, the output is identical to the R model:

In [ ]:
sleep_model_reml = fit(
  MixedModel,
  @formula(reaction ~ days + (1 | subj)),
  sleepstudy;
  REML = true
)
println(sleep_model_reml)

Linear mixed model fit by REML
 reaction ~ 1 + days + (1 | subj)
 REML criterion at convergence: 1786.4650853949372

Variance components:
            Column    Variance Std.Dev.
subj     (Intercept)  1378.1785 37.1238
Residual               960.4566 30.9912
 Number of obs: 180; levels of grouping factors: 18

  Fixed-effects parameters:
──────────────────────────────────────────────────
                Coef.  Std. Error      z  Pr(>|z|)
──────────────────────────────────────────────────
(Intercept)  251.405     9.74672   25.79    <1e-99
days          10.4673    0.804221  13.02    <1e-38
──────────────────────────────────────────────────


For comparison, here's the R model's `summary()` (truncated):

```r
sleep_model <- lmer(Reaction ~ Days + (1 | Subject), lme4::sleepstudy)
summary(sleep_model)
#> Random effects:
#>  Groups   Name        Variance Std.Dev.
#>  Subject  (Intercept) 1378.2   37.12   
#>  Residual              960.5   30.99   
#> Number of obs: 180, groups:  Subject, 18
#>
#> Fixed effects:
#>             Estimate Std. Error t value
#> (Intercept) 251.4051     9.7467   25.79
#> Days         10.4673     0.8042   13.02
```

# 3) A two-language workflow

In an ideal world, we would all simply learn Julia. The problem is, nobody has time for that.

But there's a good middle ground. We leverage the following two facts:

1) In linguistics/psychology/cogsci research, fitting (confirmatory) models is largely an *isolated* task.

2) MixedModels.jl in Julia and lme4 in R are written for the same *audiences* (primarily researchers) and *usecases* (primarily, causal inference for experimental research).

In a nutshell, the proposal is to build a workflow that uses **MixedModels.jl as a drop-in replacement for lme4**, with minimal disruption to the existing R workflow. We can imagine the following division of labor (R on the left, Julia on the right)

<img src="https://i.imgur.com/XdhoclQ.png" width="40%">

We've covered the basic syntax for fitting models in Julia (and you'll quickly pick up the rest if you already have experience with `lme4`). Now we go over the task of transferring data between R and Julia.

# 4) Demo

In the spirit of "frictionless integration", I'll demo a way of working with R and Julia simultaneously inside RStudio. No need to pick up a whole new IDE (and no need to learn Julia beyond MixedModels.jl).

**Note**: At this point, you may choose to follow along in RStudio if you've completed the setup in (1d) and can run Julia from the RStudio terminal. Otherwise, you can keep running Julia code from the notebook.

## 4a) Julia in RStudio

### **Opening Julia in RStudio**

Go to the Terminal tab of the console panel, type in "julia" and enter. This should open the Julia REPL:

```shell
               _
   _       _ _(_)_     |  Documentation: https://docs.julialang.org
  (_)     | (_) (_)    |
   _ _   _| |_  __ _   |  Type "?" for help, "]?" for Pkg help.
  | | | | | | |/ _` |  |
  | | |_| | | | (_| |  |  Version 1.10.0-rc1 (2023-11-03)
 _/ |\__'_|_|_|\__'_|  |  Official https://julialang.org/ release
|__/                   |

julia>
```

If you do not see a Terminal tab, try `Alt + Shift + R`, or search from the [command palette](https://docs.posit.co/ide/user/ide/guide/ui/command-palette.html).

### **Creating and opening Julia file from RStudio**

Simply run the following lines of R code:

```r
file.create("script.jl")
file.edit("script.jl") # Or double click from RStudio "Files" tab
```

Note that if you start Julia from RStudio terminal, the R session and the Julia session will share the same working directory, which is nice!

### **Useful keyboard shortcuts**

RStudio provides the following shortcuts to switch between console-terminal and to send code to terminal:

- **“Send Selection to Terminal”**: Sends the highlighted selection (or current line) to terminal.
  - I keybind this to `Ctrl + Alt + Enter`
- **“Move Focus to Terminal”**: Switches to the terminal tab in the console panel.
  - I keybind this to `Ctrl + Alt + RightArrow`
- **“Move Focus to Console”**: Switches to the console tab in the console panel.
  - I keybind this to `Ctrl + Alt + LeftArrow`
- **"Switch Focus Between Source and Console"**: Switches between script to console/terminal
  - I keybind this to `Alt + DownArrow`


## 4b) Getting the data from R to Julia

We can transfer data from R to Julia by simply *writing to a file from R*, and *reading in the file from Julia*.

In the case of a dataframe, we can use a csv.

So instead of fitting a model on the prepared data in R:

```r
model_data <- ...
model <- lmer(y ~ x + (1 | z), model_data)
```

You'd firstly write out the data in R:

```r
write.csv(model_data, "model_data.csv")
```

And then read it back in to fit the model in Julia:

```julia
# We use additional packaged to data from CSV into a data frame
using CSV
using DataFrames
model_data = CSV.read("model_data.csv", DataFrame)

# Fit model to data using the MixedModels package
using MixedModels
model = fit(MixedModel, @formula(y ~ x + (1 | z)), model_data)
```


---
In this notebook, we'll imagine that we've already done data processing/cleaning and have written out our file from R, to pick the modelling part back up in Julia. The data we're loading in now comes from [a prior research of mine](https://escholarship.org/uc/item/5d84n2x7) where I reported the following logistic mixed effects model:

```r
glmer(
  Accuracy ~ PitchAccent + SemanticFit + TransitivityBias +
             (1 + PitchAccent || Subject) + (1 | Item),
  data = speeded_comprehension,
  family = "binomial"
)
```

The R data frame `speeded_comprehension` has the following columns:
- `Accuracy`: 1 or 0 (response variable)
- `Subject`: Participants (grouping variable)
- `Item`: Sentences (grouping variable)
- `PitchAccent`: 2-level experimental factor, between-subjects/item
- `SemanticFit`: centered within-item norming score (continuous)
- `TransitivityBias`: centered within-item norming score (continuous)

```r
head(speeded_comprehension)
#> # A tibble: 6 × 6
#>   Accuracy Subject Item     PitchAccent SemanticFit TransitivityBias
#>      <dbl> <fct>   <chr>          <dbl>       <dbl>            <dbl>
#> 1        1 S01     Awakened           1     -0.296            -1.22
#> 2        1 S01     Calmed             1      0.0988           -0.410
#> 3        1 S01     Choked             1      1.28             -1.43
#> 4        1 S01     Dressed           -1     -0.593            -1.21
#> 5        1 S01     Failed            -1     -0.988             0.110
#> 6        1 S01     Groomed           -1     -1.09              0.989
```

In an actual workflow, we'd save this modelling-ready data out to a csv:

```r
write_csv(speeded_comprehension, "speeded_comprehension.csv")
```

For the purposes of this demo, we'll simply downloaded the file from the web into colab, to load into Julia in the next step:

In [ ]:
download("https://raw.githubusercontent.com/yjunechoe/MindCORE-julia-mixedmodels/main/speeded_comprehension.csv", "speeded_comprehension.csv")

"speeded_comprehension.csv"

## 4c) Fitting models in Julia

In [ ]:
using CSV
using DataFrames
using MixedModels

# Read CSV as a data frame
speeded_comprehension = CSV.read("speeded_comprehension.csv", DataFrame)

Row,Accuracy,Subject,Item,PitchAccent,SemanticFit,TransitivityBias
,Int64,String3,String15,Int64,Float64,Float64
1,1,S01,Awakened,-1,-0.296312,-1.22009
2,1,S01,Calmed,-1,0.0987707,-0.410233
3,1,S01,Choked,-1,1.28402,-1.42849
4,1,S01,Dressed,1,-0.592624,-1.20872
5,1,S01,Failed,1,-0.987707,0.109884
6,0,S01,Groomed,1,-1.08648,0.988955
7,0,S01,Healed,1,0.0987707,-0.371332
8,1,S01,Hides,1,0.691395,-0.159142
9,0,S01,Improved,-1,0.0987707,0.820466


In [ ]:
# Replicated model
model = fit(
  MixedModel,
  @formula(Accuracy ~ PitchAccent + SemanticFit + TransitivityBias +
                      zerocorr(1 + PitchAccent | Subject) + (1| Item)),
  speeded_comprehension,
  Bernoulli()
)

Minimizing 2    Time: 0:00:00 ( 0.15  s/it)

Minimizing 53    Time: 0:00:00 ( 7.50 ms/it)

Minimizing 104    Time: 0:00:00 ( 4.79 ms/it)

Minimizing 108    Time: 0:00:00 ( 4.72 ms/it)


|                  |    Est. |     SE |     z |      p | σ_Subject | σ_Item |
|:---------------- | -------:| ------:| -----:| ------:| ---------:| ------:|
| (Intercept)      |  1.7193 | 0.2015 |  8.53 | <1e-16 |    0.9629 | 0.6466 |
| PitchAccent      | -0.1946 | 0.0775 | -2.51 | 0.0120 |    0.1708 |        |
| SemanticFit      |  0.4456 | 0.1514 |  2.94 | 0.0032 |           |        |
| TransitivityBias | -0.2088 | 0.1598 | -1.31 | 0.1912 |           |        |


Compare to the results from R:

```r
glmer(
  Accuracy ~ PitchAccent + SemanticFit + TransitivityBias +
             (1 + PitchAccent || Subject) + (1| Item),
  data = speeded_comprehension,
  family = "binomial"
)
```

|              | Estimate (SE) | t     | p      |
|--------------|---------------|-------|--------|
| (Intercept)  |  1.72 (0.21)  |  8.36 | **>0.001** |
| PitchAccent  | –0.19 (0.08)  | –2.50 | **0.0123** |
| SemanticFit  | 0.44 (0.15)  |  2.94 | **0.0033** |
| TransitivityBias | –0.19 (0.16)  | –1.31 | 0.1910 |

Julia/MixedModels is faster and more powerful than R/lme4.*

<details>
<summary>*The fineprint</summary>
The performance gap is a combination of differences in language design (R vs. Julia), in the MEM implementation (lme4 vs. R), and in the default <a hredf="https://en.wikipedia.org/wiki/Basic_Linear_Algebra_Subprograms">BLAS</a> shipped with each language (OpenBLAS vs. XBLAS). So it's not just "R is slower", and there <em>are</em> real tradeoffs in using Julia (namely, <a href="https://en.wikipedia.org/wiki/Just-in-time_compilation">JIT</a> and pkg precompilation). There are ways of making R/lme4 faster by <a href="https://cran.r-project.org/doc/manuals/r-devel/R-admin.html#BLAS">changing some defaults</a>, but IMO this is less accessible than simply learning Julia/MixedModels.jl and using <em>its</em> defaults.
<strong>FYI</strong>, it's easy to switch out BLAS implementations in Julia: you may see further speed improvements with <a href="https://github.com/JuliaLinearAlgebra/MKL.jl">MKL</a> (for Intel cores) and <a href="https://github.com/JuliaLinearAlgebra/AppleAccelerate.jl">AppleAccelerate</a> (for MacOS)
</details>

It also lets us fit the model with maximal RE structure, which fails to converge in R:

In [ ]:
model_max = fit(
  MixedModel,
  @formula(Accuracy ~ PitchAccent + SemanticFit + TransitivityBias +
                      (1 + PitchAccent + SemanticFit + TransitivityBias | Subject) +
                      (1 + PitchAccent | Item)),
  speeded_comprehension,
  Bernoulli();
  progress = false
)
model_max

|                  |    Est. |     SE |     z |      p | σ_Subject | σ_Item |
|:---------------- | -------:| ------:| -----:| ------:| ---------:| ------:|
| (Intercept)      |  1.7645 | 0.2052 |  8.60 | <1e-17 |    1.0011 | 0.6458 |
| PitchAccent      | -0.2593 | 0.0811 | -3.20 | 0.0014 |    0.2332 | 0.0210 |
| SemanticFit      |  0.5053 | 0.1518 |  3.33 | 0.0009 |    0.1095 |        |
| TransitivityBias | -0.2557 | 0.1597 | -1.60 | 0.1095 |    0.0955 |        |


But as is often the case, this maximal model is overparameterized:

In [ ]:
issingular(model_max)

true

In [ ]:
VarCorr(model_max)

|         | Column           |  Variance |   Std.Dev | Corr. |       |       |
|:------- |:---------------- | ---------:| ---------:| -----:| -----:| -----:|
| Subject | (Intercept)      | 1.0021195 | 1.0010592 |       |       |       |
|         | PitchAccent      | 0.0543673 | 0.2331680 | -0.62 |       |       |
|         | SemanticFit      | 0.0119795 | 0.1094507 | +1.00 | -0.55 |       |
|         | TransitivityBias | 0.0091180 | 0.0954881 | -0.93 | +0.86 | -0.90 |
| Item    | (Intercept)      | 0.4170966 | 0.6458302 |       |       |       |
|         | PitchAccent      | 0.0004406 | 0.0209901 | -1.00 |       |       |


Turns out that only a modest increase in complexity of the model (just including the extra correlation term) is warranted by the data:

In [ ]:
model_final = fit(
  MixedModel,
  @formula(Accuracy ~ PitchAccent + SemanticFit + TransitivityBias +
                      (1 + PitchAccent | Subject) +
                      (1 | Item)),
  speeded_comprehension,
  Bernoulli();
  progress=false
)

|                  |    Est. |     SE |     z |      p | σ_Subject | σ_Item |
|:---------------- | -------:| ------:| -----:| ------:| ---------:| ------:|
| (Intercept)      |  1.7390 | 0.2046 |  8.50 | <1e-16 |    0.9881 | 0.6518 |
| PitchAccent      | -0.2471 | 0.0803 | -3.08 | 0.0021 |    0.2220 |        |
| SemanticFit      |  0.4474 | 0.1523 |  2.94 | 0.0033 |           |        |
| TransitivityBias | -0.2084 | 0.1607 | -1.30 | 0.1947 |           |        |


In [ ]:
VarCorr(model_final)

|         | Column      |  Variance |  Std.Dev | Corr. |
|:------- |:----------- | ---------:| --------:| -----:|
| Subject | (Intercept) |  0.976389 | 0.988124 |       |
|         | PitchAccent |  0.049276 | 0.221982 | -0.61 |
| Item    | (Intercept) |  0.424883 | 0.651831 |       |


In [ ]:
issingular(model_final)

false

For fun, the more complex (two-way) **interaction model** that I wish I'd reported:

In [ ]:
@time model_interactions = fit(
  MixedModel,
  @formula(Accuracy ~ PitchAccent + SemanticFit + TransitivityBias +
                      PitchAccent & SemanticFit +
                      PitchAccent & TransitivityBias +
                      SemanticFit & TransitivityBias +
                      (1 + PitchAccent | Subject) +
                      (1 | Item)),
  speeded_comprehension,
  Binomial();
  progress=false
)
println(model_interactions)

  6.246394 seconds (4.62 M allocations: 293.772 MiB, 3.26% gc time, 82.89% compilation time)
Generalized Linear Mixed Model fit by maximum likelihood (nAGQ = 1)
  Accuracy ~ 1 + PitchAccent + SemanticFit + TransitivityBias + PitchAccent & SemanticFit + PitchAccent & TransitivityBias + SemanticFit & TransitivityBias + (1 + PitchAccent | Subject) + (1 | Item)
  Distribution: Bernoulli{Float64}
  Link: LogitLink()

   logLik    deviance     AIC       AICc        BIC    
  -630.5990  1261.1981  1283.1981  1283.3832  1341.1792

Variance components:
           Column   Variance Std.Dev.   Corr.
Subject (Intercept)  0.969970 0.984871
        PitchAccent  0.044832 0.211736 -0.54
Item    (Intercept)  0.301111 0.548736

 Number of obs: 1438; levels of grouping factors: 61, 24

Fixed-effects parameters:
───────────────────────────────────────────────────────────────────────
                                     Coef.  Std. Error      z  Pr(>|z|)
────────────────────────────────────────────────────

In [ ]:
issingular(model_interactions)

false

Omnibus test on the nested models: adding two-way interactions significantly improve model fit:

In [ ]:
# Equivalent to R's `anova()`
MixedModels.likelihoodratiotest(model_final, model_interactions)

|                                                                                                                                                                                                        | model-dof | deviance |  χ² | χ²-dof | P(>χ²) |
|:------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ | ---------:| --------:| ---:| ------:|:------ |
| Accuracy ~ 1 + PitchAccent + SemanticFit + TransitivityBias + (1 + PitchAccent \| Subject) + (1 \| Item)                                                                                               |         8 |     1269 |     |        |        |
| Accuracy ~ 1 + PitchAccent + SemanticFit + TransitivityBias + PitchAccent & SemanticFit + PitchAccent & TransitivityBias + SemanticFit & TransitivityBias + (1 + PitchAccent \| Subject) + (1 \| Item) |        11 |     1261 |   8 |      3 | 0.0441 |


## 4e) Getting model from Julia to R

Assume that we're satisfied with our model, `model_final`. How do we do more with it outside of Julia?

Admittedly, it's a bit difficult to transfer *models* back and forth because they have slightly different representations in R vs. Julia.

But often times, we don't really care about the internal details of the model. Julia has utilities for [extracting](https://juliastats.org/MixedModels.jl/stable/constructors/#Extractor-functions) numbers off of models, which you can then write out to a csv to read back in R:

In [ ]:
# `coeftable()` extracts fixed effects from the summary
coef_table = coeftable(model_final);
println(coef_table)

────────────────────────────────────────────────────────
                      Coef.  Std. Error      z  Pr(>|z|)
────────────────────────────────────────────────────────
(Intercept)        1.73901    0.204637    8.50    <1e-16
PitchAccent       -0.247086   0.0803251  -3.08    0.0021
SemanticFit        0.447363   0.152268    2.94    0.0033
TransitivityBias  -0.208408   0.16072    -1.30    0.1947
────────────────────────────────────────────────────────


In [ ]:
# This can be turned into tabular data
using DataFrames
coef_datatable = DataFrame(coef_table)
println(coef_datatable)

4×5 DataFrame
 Row │ Name              Coef.      Std. Error  z         Pr(>|z|)    
     │ String            Float64    Float64     Float64   Float64     
─────┼────────────────────────────────────────────────────────────────
   1 │ (Intercept)        1.73901    0.204637    8.49802  1.92858e-17
   2 │ PitchAccent       -0.247086   0.0803251  -3.07607  0.00209746
   3 │ SemanticFit        0.447363   0.152268    2.93801  0.00330332
   4 │ TransitivityBias  -0.208408   0.16072    -1.29672  0.194729


In [ ]:
# The tabular data can then be saved out to csv
using CSV
CSV.write("model_output.csv", coef_datatable)

"model_output.csv"

---

In case we still *do* really care about getting the model *object* back in R, there are also ways of doing that too. It's not a full conversion, but you can do just enough reconstruction to support ex: [marginaleffects](https://marginaleffects.com/), [emmeans](https://github.com/rvlenth/emmeans), [lmerTest](https://cran.r-project.org/web/packages/lmerTest/index.html), [afex](https://cran.r-project.org/web/packages/afex/index.html), etc. back in R.

Colab doesn't have the necessary infrastructure to support this, but this is the code for that (requires `JellyMe4` and `RCall` to be installed and set up in Julia). I'll demo this on my laptop in RStudio.

In Julia:

```julia
# 1) Load packages
using RCall
using JellyMe4

# 2) Send model/data over to R
julia_model = (model, model_data)
@rput julia_model

# 3) Inspect/write model from linked R session (via RCall)
R"julia_model"
R"saveRDS(julia_model, 'julia_model.rds')"
```

In R:

```r
# 4) Read model in RStudio R session
julia_model <- readRDS("julia_model.rds")
```

# Fin.

We've come a long way and we're in good hands

<img src="https://i.imgur.com/fbH6COS.png" width="25%">

Acknowledgments:

- [SMLP2023](https://vasishth.github.io/smlp2023/) workshop instructors: Phillip Alday, Douglass Bates, Reinhold Kliegl.

- [MindCORE](https://mindcore.sas.upenn.edu/) for hosting/organizing this event and for the activity grant which allowed me to attend the above workshop.

# Further readings/resources

- SMLP workshop materials: [SMLP2023: Advanced Methods in Frequentist Statistics with Julia](https://repsychling.github.io/SMLP2023/)

- Book by the workshop instructors: [Embrace Uncertainty](https://embraceuncertaintybook.com/)

- Documentation websites for [MixedModels.jl](https://juliastats.org/MixedModels.jl/stable/) and related packages:

  - [MixedModelsSim.jl](https://repsychling.github.io/MixedModelsSim.jl/stable/): for simulation and power analysis
  - [MixedModelsMakie.jl](https://palday.github.io/MixedModelsMakie.jl/dev/api/): for visualization
  - [RCall](https://juliainterop.github.io/RCall.jl/stable/installation/): for running R in Julia
  - [JellyMe4.jl](https://github.com/palday/JellyMe4.jl): for converting MixedModels.jl models to R lme4 models
  - [Effects.jl](https://docs.juliahub.com/Effects/qason/0.1.1/): for predictions and post-hoc hypothesis tests

- For learning Julia:

  - Julia manual's ["Noteworthy differences from R"](https://docs.julialang.org/en/v1.9/manual/noteworthy-differences/#Noteworthy-differences-from-R)
  - [Julia Data Science](https://juliadatascience.io/): The [R4DS book](https://r4ds.hadley.nz/) equivalent for Julia
  - [Tidier.jl](https://github.com/TidierOrg/Tidier.jl): Implementation of the tidyverse in Julia
